# GD-1

In [16]:
import os
import glob
import pathlib
import warnings

import astropy.coordinates as coord
from astropy.io import fits
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from pyia import GaiaData
import gala.coordinates as gc

from astroquery.gaia import Gaia
Gaia.login(credentials_file=os.path.expanduser('~/.gaia/archive.login'))

from stream_helpers import q_base

OK


In [6]:
DATA_PATH = pathlib.Path('../data/').resolve()
DATA_PATH.mkdir(exist_ok=True)

In [ ]:
g = GaiaData('/Users/apricewhelan/projects/gd1-dr2/data/gd1-master.fits')
probs = np.load('/Users/apricewhelan/projects/gd1-dr2/output/pm-model-phi1phi2prob.npy')

In [ ]:
c = g.get_skycoord(distance=False)
gd1_c = c.transform_to(gc.GD1Koposov10)

In [ ]:
mask = probs[2] > 0.2

fig, ax = plt.subplots(1, 1, figsize=(12, 2))
ax.plot(gd1_c.phi1.value[mask], 
        gd1_c.phi2.value[mask], 
        ls='none', alpha=0.2, mew=0, marker='o', ms=1.5)

In [7]:
queries = []
jobs = {}
for l in np.arange(-110, 30, 10):
    print(l)
    
    fn = DATA_PATH / f'gd1/gd1_{l:.0f}.fits'
    fn.parent.mkdir(exist_ok=True)
    
    if fn.exists():
        print(f'{fn} exists...skipping')
        continue

    corners = coord.SkyCoord(phi1=[l, l, l+10, l+10] * u.deg,
                             phi2=[-10, 5, 5, -10] * u.deg,
                             frame=gc.GD1Koposov10())
    corners_icrs = corners.transform_to(coord.ICRS)
    
    q = q_base.format(corners_icrs)
        
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        job = Gaia.launch_job_async(q, name='GD1_EDR3_{0}'.format(l),
                                    background=True)
    jobs[fn] = job

-110
-100
-90
-80
-70
-60
-50
-40
-30
-20
-10
0
10
20


The full footprint to try a PS1 query:

In [10]:
corners = coord.SkyCoord(phi1=[-110, -110, 30, 30] * u.deg,
                         phi2=[-10, 5, 5, -10] * u.deg,
                         frame=gc.GD1Koposov10())
corners_icrs = corners.transform_to(coord.ICRS)

In [11]:
for fn, job in jobs.items():
    tbl = job.get_results()

    for c in tbl.colnames: # hack to make sure object arrays are string
        if tbl[c].dtype == object:
            tbl[c] = np.array(tbl[c]).astype(str)
            
    tbl.write(fn, overwrite=True)

KeyboardInterrupt: 

## Combine tables

In [12]:
from numpy.lib.recfunctions import stack_arrays

In [19]:
all_filename = DATA_PATH / 'gd1-all.fits'
if not all_filename.exists():
    arrs = []
    for filename in glob.glob(str(DATA_PATH / 'gd1/gd1_*')):
        arrs.append(np.array(fits.getdata(filename)))
    arr = stack_arrays(arrs, asrecarray=True, usemask=False)
    
    t = at.Table(arr)
    t.write(all_filename)